# Scaling the data set

## Table of contents

### [1. Libraries and Data importations](#1)
### [2. Data Preparation](#2)

-[2.1 Data examination](#2.1)

-[ 2.2 Data cleaning](#2.2)

### [3. Scaling](#3)

-[3.1 Subsetting numerical variables](#3.1)

-[3.2 Scaling](#3.2)

### [4. Saving scaled df](#4)

-[4.1 Adding dates back](#4.1)

-[4.2 Saving as PKL](#4.2)

## 1. Libraries and Data importations
<div id='1'></div>

In [162]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sklearn
from sklearn.preprocessing import StandardScaler


##to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Creating paths to the data folder of the project and to the folder for saving the charts
path_data = r"C:\Users\dacol\Documents\Data Project - careerfoundry\ClimateWins_ML\02 Data"


# Importing the latest merged dataframe
df = pd.read_csv(os.path.join(path_data,'Original Data','Dataset-weather-prediction-dataset-processed.csv'), sep =',')


In [163]:
df.head(10)

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4
5,19600106,1,1,2.1,0.78,1.018,0.76,0.00,0,7.7,...,5,0.76,1.0326,0.33,0.00,0,0.9,4.1,-0.6,8.7
6,19600107,1,3,2.1,0.87,1.018,0.66,0.00,0,5.4,...,7,0.73,1.0230,0.42,0.00,0,2.2,7.9,4.4,11.4
7,19600108,1,8,2.1,0.95,1.018,0.19,0.09,0,0.0,...,2,0.72,1.0203,0.50,0.00,0,6.2,8.5,6.7,10.3
8,19600109,1,8,2.1,0.98,1.018,0.19,0.56,0,0.0,...,5,0.67,1.0243,0.14,0.00,0,0.1,6.8,5.8,7.8
9,19600110,1,3,2.1,0.74,1.018,0.38,0.01,5,1.1,...,1,0.66,1.0307,0.49,0.00,0,6.4,1.8,0.2,5.0


## 2. Data preparation
<div id='2'></div>

### 2.1 Data examination
<div id='2.1'></div>

In [166]:
df.info('all') #dates are integers and should be dates

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22950 entries, 0 to 22949
Data columns (total 170 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    DATE                         int64  
 1    MONTH                        int64  
 2    BASEL_cloud_cover            int64  
 3    BASEL_wind_speed             float64
 4    BASEL_humidity               float64
 5    BASEL_pressure               float64
 6    BASEL_global_radiation       float64
 7    BASEL_precipitation          float64
 8    BASEL_snow_depth             int64  
 9    BASEL_sunshine               float64
 10   BASEL_temp_mean              float64
 11   BASEL_temp_min               float64
 12   BASEL_temp_max               float64
 13   BELGRADE_cloud_cover         int64  
 14   BELGRADE_humidity            float64
 15   BELGRADE_pressure            float64
 16   BELGRADE_global_radiation    float64
 17   BELGRADE_precipitation       float64
 18   BELGRADE_sunshine       

The data set is about weather metrics for a selection of cities in Europe, one for each country as it seems.

In [168]:
df.isnull().sum() #no null values !

DATE                   0
MONTH                  0
BASEL_cloud_cover      0
BASEL_wind_speed       0
BASEL_humidity         0
                      ..
VALENTIA_snow_depth    0
VALENTIA_sunshine      0
VALENTIA_temp_mean     0
VALENTIA_temp_min      0
VALENTIA_temp_max      0
Length: 170, dtype: int64

In [169]:
df.describe() #no values are looking odd, some of the units are to be precised (celsius, bars, etc)

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
count,2.295000e+04,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,...,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000,22950.000000
mean,1.990984e+07,6.509630,5.410763,2.120462,0.758554,1.018013,1.345244,0.222305,0.359564,4.592222,...,5.723355,0.825824,1.014242,1.134490,0.414698,0.001220,3.460989,10.724257,7.901904,13.515752
std,1.813833e+05,3.443672,2.406115,0.732625,0.110699,0.006543,0.931158,0.498995,2.006231,4.310808,...,1.630313,0.071121,0.010727,0.848813,0.844943,0.049383,3.329432,3.328727,3.659393,3.477373
min,1.960010e+07,1.000000,0.000000,0.000000,0.350000,0.974700,0.010000,0.000000,-2.000000,0.000000,...,0.000000,0.380000,0.955100,0.020000,0.000000,0.000000,0.000000,-3.500000,-7.300000,-1.500000
25%,1.975092e+07,4.000000,4.000000,2.100000,0.680000,1.015800,0.540000,0.000000,0.000000,0.400000,...,5.000000,0.790000,1.009400,0.400000,0.010000,0.000000,0.500000,8.800000,6.100000,11.300000
50%,1.991060e+07,7.000000,6.000000,2.100000,0.770000,1.018000,1.130000,0.000000,0.000000,3.600000,...,6.000000,0.820000,1.014200,0.970000,0.280000,0.000000,3.400000,10.700000,7.900000,13.500000
75%,2.007021e+07,9.000000,7.000000,2.100000,0.840000,1.020100,2.070000,0.220000,0.000000,7.900000,...,7.000000,0.870000,1.020900,1.700000,0.410000,0.000000,4.800000,13.000000,10.300000,15.700000
max,2.022103e+07,12.000000,8.000000,16.300000,1.000000,1.045200,4.560000,8.500000,49.000000,16.800000,...,8.000000,1.000000,1.046300,3.980000,90.000000,3.000000,15.800000,23.600000,19.500000,28.400000


Not much data cleaning is required, except for the dates

### 2.2 Data cleaning
<div id='2.2'></div>

In [172]:
df['DATE']= pd.to_datetime(df['DATE'],format='%Y%m%d') #changing the 'DATE' variable to a datetime format

In [173]:
df['DATE'].info() #change succesfull !

<class 'pandas.core.series.Series'>
RangeIndex: 22950 entries, 0 to 22949
Series name: DATE
Non-Null Count  Dtype         
--------------  -----         
22950 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 179.4 KB


## 3. Scaling
<div id='3'></div>

### 3.1 Subsetting numerical variables
<div id='3.1'></div>

In [176]:
df_sub = df.loc[:,'BASEL_cloud_cover':'VALENTIA_temp_max']
df_sub.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


### 3.2 Scaling
<div id='3.2'></div>

In [178]:
scaler = StandardScaler() #creating the scaler object
columns = df_sub.columns #isolating the columns names to re-use them on the scaled df

In [179]:
#applying the standard scaling object to the subset

df_scaled = pd.DataFrame(scaler.fit_transform(df_sub), columns=columns)
df_scaled.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,-0.528623,-0.845652,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237
1,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,-0.582946,-0.462450,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141
2,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,-0.257010,-0.186545,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078
3,-1.001953,-0.02793,1.458455,-0.001949,-0.821838,-0.445514,-0.179228,-0.114186,-0.555784,-0.385810,...,0.783085,0.480538,0.387574,-1.183432,0.669056,-0.024706,-1.039536,-0.998679,-0.164486,-0.838511
4,0.244897,-0.02793,1.729466,-0.001949,-0.746661,-0.164944,-0.179228,0.187388,-1.003946,-1.075573,...,-1.670486,-0.363113,1.729970,-0.794645,-0.490810,-0.024706,0.672505,-1.509396,-1.339569,-1.471186


In [180]:
df_scaled.describe()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
count,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,...,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04,2.295000e+04
mean,-1.975278e-16,-8.817541e-16,6.935145e-16,-1.690442e-15,-2.179617e-16,-6.997066e-17,-4.953675e-18,-3.962940e-17,1.882396e-16,4.953675e-17,...,-1.486102e-16,-1.709018e-15,-1.800104e-14,-1.368453e-16,-1.238419e-18,4.953675e-18,1.733786e-17,4.458307e-17,-1.535639e-16,-1.356068e-16
std,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,...,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00
min,-2.248803e+00,-2.894398e+00,-3.690749e+00,-6.619945e+00,-1.433992e+00,-4.455141e-01,-1.176144e+00,-1.065304e+00,-3.923787e+00,-4.539717e+00,...,-3.510664e+00,-6.268665e+00,-5.513376e+00,-1.313028e+00,-4.908101e-01,-2.470616e-02,-1.039536e+00,-4.273275e+00,-4.154305e+00,-4.318225e+00
25%,-5.863365e-01,-2.793014e-02,-7.096309e-01,-3.381979e-01,-8.647959e-01,-4.455141e-01,-1.792277e-01,-9.725122e-01,-7.730748e-01,-7.536835e-01,...,-4.437007e-01,-5.037211e-01,-4.514238e-01,-8.653338e-01,-4.789747e-01,-2.470616e-02,-8.893570e-01,-5.780885e-01,-4.924159e-01,-6.372051e-01
50%,2.448969e-01,-2.793014e-02,1.034013e-01,-1.948634e-03,-2.311624e-01,-4.455141e-01,-1.792277e-01,-2.301758e-01,4.176451e-02,2.804803e-02,...,1.696919e-01,-8.189596e-02,-3.958379e-03,-1.937928e-01,-1.594196e-01,-2.470616e-02,-1.831857e-02,-7.287352e-03,-5.203543e-04,-4.529850e-03
75%,6.605137e-01,-2.793014e-02,7.357597e-01,3.190165e-01,7.783554e-01,-4.618532e-03,-1.792277e-01,7.673387e-01,7.887005e-01,7.944515e-01,...,7.830846e-01,6.211460e-01,6.206288e-01,6.662509e-01,-5.559789e-03,-2.470616e-02,4.021827e-01,6.836824e-01,6.553403e-01,6.281454e-01
max,1.076130e+00,1.935485e+01,2.181150e+00,4.155315e+00,3.452504e+00,1.658909e+01,2.424521e+01,2.831962e+00,2.554186e+00,2.465211e+00,...,1.396477e+00,2.449055e+00,2.988467e+00,3.352415e+00,1.060275e+02,6.072597e+01,3.706122e+00,3.868152e+00,3.169473e+00,4.280407e+00


All weather metrics have a standard deviation equal to 1, meaning the scaling function worked succesfully!

## 4. Saving scaled df
<div id='4'></div>

### 4.1 Adding dates back
<div id='4.1'></div>

In [203]:
df_dates = df[['DATE', 'MONTH']] #isolating the date and month columns

In [205]:
#concate the dates with the scaled values

df_final = pd.concat([df_dates.reset_index(drop=True), df_scaled.reset_index(drop =True)], axis =1)
df_final.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,1960-01-01,1,0.660514,-0.02793,0.826097,-0.001949,-1.101066,-0.265148,-0.179228,-0.902918,...,-0.443701,0.761754,-1.299744,-0.806427,-0.088407,-0.024706,0.372147,-0.668215,-0.519743,-0.752237
1,1960-01-02,1,0.244897,-0.02793,0.735760,-0.001949,-1.058108,1.658760,-0.179228,-0.810126,...,0.783085,1.183580,-1.262455,-1.042055,0.503361,-0.024706,-0.829285,-0.548046,-0.629054,-0.407141
2,1960-01-03,1,1.076130,-0.02793,1.277781,-0.001949,-1.251420,0.155707,-0.179228,-1.065304,...,0.783085,1.183580,-0.432779,-1.136306,-0.396127,-0.024706,-1.009500,-0.067372,0.054135,-0.177078
3,1960-01-04,1,-1.001953,-0.02793,1.458455,-0.001949,-0.821838,-0.445514,-0.179228,-0.114186,...,0.783085,0.480538,0.387574,-1.183432,0.669056,-0.024706,-1.039536,-0.998679,-0.164486,-0.838511
4,1960-01-05,1,0.244897,-0.02793,1.729466,-0.001949,-0.746661,-0.164944,-0.179228,0.187388,...,-1.670486,-0.363113,1.729970,-0.794645,-0.490810,-0.024706,0.672505,-1.509396,-1.339569,-1.471186


### 4.2 Saving as PKL
<div id='4.2'></div>

In [187]:
df_final.to_pickle(os.path.join(path_data, 'Prepared Data','Dataset_scaled.pkl'))